Scrape www.healthgrades.com


In [0]:
#Import
import numpy as np
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import re
import time
from time import sleep
from tqdm import tqdm

In [2]:
#NOTE: This cell may not work in Google Colab, since it opens a new Chrome Window
#Use headless browser to render doctor URL list
from selenium import webdriver
pathy='Path_to_chromedriver.exe' #Path to chromedriver.exe driver

#Scrape doctor URL List
pagelimit=525 #Number of webpages in directory 

#Internal Medicine Doctors in California
for page in pages:
    urlstart = "https://www.healthgrades.com/usearch?what=Internal%20Medicine&entityCode=PS412&where=CA&pageNum="
    urlend = "&sort.provider=bestmatch&state=CA"
    url=urlstart+page+urlend
    driver = webdriver.Chrome(pathy)
    driver.get(url);
    time.sleep(3)
    doclist=[]
    for a in driver.find_elements_by_xpath('.//a'):
        b = a.get_attribute('href')
        try: 
            if b.startswith("https://www.healthgrades.com/physician/"):
                doclist=np.append(doclist,b)
        except AttributeError:
            pass
    driver.quit()
    np.savetxt(page+".txt",doclist, fmt='%s')

ModuleNotFoundError: ignored

In [0]:
#Combine all pages into one text file
pages=[]
for i in range(1,pagelimit):
    pages=np.append(pages,str(i))

pagenames=[]
for page in pages:
    pagenames=np.append(pagenames,page+".txt")

ofile="internal_CA.txt"
filenames = pagenames
with open(ofile, 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

In [0]:
#Strip "leave-review" from doctor URLs
def deleteLine():
    fn = 'internal_CA.txt'
    f = open(fn)
    output = []
    str="leave-review"
    for line in f:
        if not 'leave-review' in line:
            output.append(line)
    f.close()
    f = open(fn, 'w')
    f.writelines(output)
    f.close()
deleteLine()

FileNotFoundError: ignored

In [0]:
#Define text find function
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches

def getLinks(url):
    html_page = requests.get(url).content
    soup = BeautifulSoup(html_page)
    links = []

    for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
        links.append(link.get('href'))

    return links

In [0]:
#Load URLs
with open('internal_CA.txt', 'r') as f:
    x = f.readlines()
docnames = [line.rstrip() for line in x]    

In [8]:
#Scrape data from individual doctor pages
startdoc=0 #To resume after interruption
dics=[]
maxreviews=20 #Maximum Patient reviews per doctor
for docurl in tqdm(docnames[startdoc:]): 
    url = docurl
    dic={}
    comments1=url+"/comments"

    r = requests.get(url).content
    soup = BeautifulSoup(r)
    alltext=soup.get_text()

    r2 = requests.get(comments1).content
    soup2 = BeautifulSoup(r2)
    alltext2=soup2.get_text()

    comcity=list(find_all(alltext,"locationCity"))
    f1=alltext[comcity[0]+14:comcity[0]+200]
    f2=f1.split(':')[0].split(',')[0].replace(u'\u0022',"")
    city=f2

    com=list(find_all(alltext,"responseCount"))
    findy=alltext[com[0]:com[0]+100]
    findy2=findy.split("text")[0].split(":")
    responseCount=float(findy2[1].split(',')[0])
    reviewCount=float(findy2[2].split(',')[0])
    actualScore=float(findy2[3].split(',')[0])
    roundedScore=float(findy2[4].split(',')[0])
    com2=list(find_all(alltext,"providerName"))
    findy3=alltext[com2[0]+15:com2[0]+100]
    providerName=findy3.split("providerPossessiveName")[0][:-3]
    
    try:
        comage=list(find_all(alltext,"Age"))
        age=alltext[int(comage[0]+4):int(comage[0])+6]
    except:
        pass
    
    comgen=list(find_all(soup.get_text(),"genderString"))
    gender=alltext[comgen[0]+15:comgen[0]+16]
        
    com3=list(find_all(alltext2,'Submitted on'))
    com4=list(find_all(alltext2,'Author'))
    
    try:
        comlang=list(find_all(alltext,"languages"))[1]
        lang=alltext[comlang+12:comlang+50].split("],")[0]
        langs=lang.replace(u'\u0022',"").split(",")
    except:
        pass
    
    try:
        charlimit=1000
        allstars=[]
        texts=[]
        for patient in range(int(len(com4))):
            dateSub=alltext2[com3[patient]+13:com3[patient]+29]
            stars=alltext2[com4[patient]-1:com4[patient]]
            allstars=np.append(allstars,stars)
            texty=alltext2[com3[patient]+29:com3[patient]+charlimit].split("Author:")[0][:-1]
            texts=np.append(texts,texty)
        ids=url.split("-")[-1]
    except:
        pass
    
    try:
        dic={'Doctor':providerName,"Age":int(age),"Gender":gender,"Responses":int(responseCount),
         "Reviews":int(reviewCount),"Actual_Score":np.around(actualScore,4),"Rounded_Score":roundedScore,
         "URL":url,"ID":ids,"City":city}
    except: 
        pass
    
    try:
        dic["Lang1"]=langs[0]
    except: 
        pass
    try:
        dic["Lang2"]=langs[1]
    except: 
        pass
    try:
        dic["Lang3"]=langs[2]
    except: 
        pass
    try:
        dic["Lang4"]=langs[3]
    except: 
        pass
    
    assert len(texts)==len(allstars)
    maxpatients=min(maxreviews,int(reviewCount))

    try:  
      for patient in range(maxpatients):
          dic["Stars"+str(patient).zfill(2)]=allstars[patient]
          dic["Text"+str(patient).zfill(2)]=texts[patient]
    except: 
      pass
    
    dics.append(dic)

100%|██████████| 10352/10352 [1:38:10<00:00,  1.88it/s]


In [0]:
#Export to Excel File
df = pd.DataFrame.from_dict(dics)
dforder=df[['Actual_Score',	'Age',	'Doctor',	'Gender',	'ID',	'Lang1', 'Lang2',	'Lang3', 'Lang4', 'Responses',	'Reviews',	'Rounded_Score',	'Stars00',	'Stars01',	'Stars02',	'Stars03',	'Stars04',	'Stars05',	'Stars06',	'Stars07',	'Stars08',	'Stars09',	'Stars10',	'Stars11',	'Stars12',	'Stars13',	'Stars14',	'Stars15',	'Stars16',	'Stars17',	'Stars18',	'Stars19',	'Text00',	'Text01',	'Text02',	'Text03',	'Text04',	'Text05',	'Text06',	'Text07',	'Text08',	'Text09',	'Text10',	'Text11',	'Text12',	'Text13',	'Text14',	'Text15',	'Text16',	'Text17',	'Text18',	'Text19',	'URL',	'City']]
dforder.to_excel("Internal_CA.xlsx")